In [ ]:
import numpy as np

from scipy.stats import norm
from gs_quant.common import Currency, OptionType
from gs_quant.markets import PricingContext
from gs_quant.instrument import FXOption
from gs_quant.session import GsSession, Environment
from gs_quant import risk

In [ ]:
# external users should substitute their client id and secret
GsSession.use(Environment.PROD, client_id=None, client_secret=None, scopes=('run_analytics',))

In [ ]:
cross = "USDJPY"
premium_currency = Currency.JPY

In [ ]:
option = FXOption(
    cross,
    expiration_date="2y",
    option_type=OptionType.Call,
    strike_price="25d",
    premium_currency=premium_currency,
    premium_payment_date="spot",
    notional_amount=10e6,
    notional_currency=Currency.USD,
    premium=0,
)
option.resolve()

### FXGamma
The only gamma measure currently available for FX is `FXGamma`. It represents the change in delta - wing delta in the underlying currency, scaled by the notional amount - for a 100% move in spot.

This measure is computed using a central finite difference approximation, where spot shifts for repricing are done under the sticky-delta market model.

In can be used in the following ways:

In [ ]:
gamma = option.calc(risk.FXGamma)
spot = option.calc(risk.FXSpot)
delta = option.calc(risk.FXCalcDelta) * option.notional_amount
print(f"""
Spot    : {spot:.4f}
FXGamma (change in wing delta for a 100% move in spot) : {gamma:,.0f} {cross[3:]} / {cross}
FXDelta : {delta:,.0f} {cross[:3]}
""")

#### 1) Computing change in delta for a unit spot change
Shock spot by 1 and recompute delta; the difference is approximated by `FXGamma / FXSpot`

In [ ]:
mkt_shock = risk.MarketDataShockBasedScenario(
    shocks={risk.MarketDataPattern('FX', 'JPY/USD'): risk.MarketDataShock(risk.MarketDataShockType.Absolute, 1)}
)
with mkt_shock:
    shocked_spot = option.calc(risk.FXSpot)
    shocked_delta = option.calc(risk.FXCalcDelta) * option.notional_amount

shocked_spot

In [ ]:
print(f"""
Shocked Delta    : {shocked_delta:,.0f}
Delta Diff : {shocked_delta - delta:,.0f}
FXGamma/spot  : {gamma / spot:,.0f}
""")

#### 2) Computing change in delta for a percentage spot change
Shock spot by X% and recompute delta; the difference is approximated by `FXGamma * X / 100`

In [ ]:
shock_size = 0.01
mkt_shock = risk.MarketDataShockBasedScenario(
    shocks={
        risk.MarketDataPattern('FX', 'JPY/USD'): risk.MarketDataShock(risk.MarketDataShockType.Proportional, shock_size)
    }
)
with mkt_shock:
    shocked_spot = option.calc(risk.FXSpot)
    shocked_delta = option.calc(risk.FXCalcDelta) * option.notional_amount

shocked_spot

In [ ]:
print(f"""
Shocked Delta    : {shocked_delta:,.0f}
Delta Diff : {shocked_delta - delta:,.0f}
FXGamma * {shock_size}  : {gamma * shock_size:,.0f}
""")

In [ ]:
shock_size = 0.0001  # small bump size approximates the gamma FDM calculation, so the difference will match FXGamma more closely now
mkt_shock = risk.MarketDataShockBasedScenario(
    shocks={
        risk.MarketDataPattern('FX', 'JPY/USD'): risk.MarketDataShock(risk.MarketDataShockType.Proportional, shock_size)
    }
)
with mkt_shock:
    shocked_spot = option.calc(risk.FXSpot)
    shocked_delta = option.calc(risk.FXCalcDelta) * option.notional_amount

shocked_spot

In [ ]:
print(f"""
Shocked Delta    : {shocked_delta:,.0f}
Delta Diff : {shocked_delta - delta:,.0f}
FXGamma * {shock_size}  : {gamma * shock_size:,.0f}
""")

#### 3) Validating raw gamma (FXGamma/FXSpot) against Black-Scholes gamma

In [ ]:
raw_gamma = gamma / spot
print(f"""
Raw Gamma (scaled by notional): {raw_gamma:,.0f}
""")

In [ ]:
def black_sholes_raw_gamma(spot, strike, T, df_1, df_2, volatility):
    r_1 = -np.log(df_1) / T
    r_2 = -np.log(df_2) / T
    d1 = (np.log(spot / strike) + (r_2 - r_1 + 0.5 * volatility**2) * T) / (volatility * np.sqrt(T))
    n_prime_d1 = norm.pdf(d1)
    bs_gamma = (np.exp(-r_1 * T) * n_prime_d1) / (spot * volatility * np.sqrt(T))
    return float(bs_gamma)


vol = option.calc(risk.FXAnnualImpliedVol)
df_jpy = option.calc(risk.FXDiscountFactorOver)
df_usd = option.calc(risk.FXDiscountFactorUnder)

In [ ]:
bs_raw_gamma = black_sholes_raw_gamma(spot, option.strike_price, 2.0, df_usd, df_jpy, vol)
print(f"""
Black-Scholes Raw Gamma (scaled by notional): {bs_raw_gamma * option.notional_amount:,.0f}
""")

#### 4) Computing Gamma PnL of holding the option for one day
Raw gamma can be used in the Taylor series expansion term to derive Gamma PnL in the Overlying currency.

In [ ]:
spot_price = option.calc(risk.FairPremium)  # in JPY, as option has premium_currency = JPY
spot_shocks = [i / 100 for i in range(-8, 12)]
prices = []
# wrap in PricingContext to send calculations in parallel
with PricingContext():
    for spot_shock in spot_shocks:
        with risk.MarketDataShockBasedScenario(
            shocks={
                risk.MarketDataPattern('FX', 'JPY/USD'): risk.MarketDataShock(
                    risk.MarketDataShockType.Proportional, spot_shock
                )
            }
        ):
            prices.append(option.calc(risk.FairPremium))
# prices are in JPY, as option has premium_currency = JPY
prices = [p.result() for p in prices]
ds_array = [spot * shock for shock in spot_shocks]
spots = [float(spot) + ds for ds in ds_array]

Delta is in the notional currency (USD), so `delta * ds` is in JPY, like `spot_price`

In [ ]:
prices_delta_approx = [spot_price + float(delta * ds) for ds in ds_array]

`gamma` is in the underlying currency (USD) and `raw_gamma = gamma / spot`.

Therefore `raw_gamma * ds` is in the underlying currency and `raw_gamma * (ds ** 2)` is in the overlying currency (JPY) like the rest of the terms.

In [ ]:
prices_delta_gamma_approx = [spot_price + float(delta * ds) + float(0.5 * raw_gamma * ds**2) for ds in ds_array]

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter

prices_millions = [p / 1e6 for p in prices]
prices_delta_approx_millions = [p / 1e6 for p in prices_delta_approx]
prices_delta_gamma_approx_millions = [p / 1e6 for p in prices_delta_gamma_approx]

plt.figure(figsize=(12, 8))

plt.plot(spots, prices_millions, '-', linewidth=2.5, label='Actual Option Price', color='#2E86AB', zorder=3)
plt.plot(
    spots,
    prices_delta_approx_millions,
    '--',
    linewidth=2,
    label='Delta Approximation (Linear)',
    color='#A23B72',
    alpha=0.8,
    zorder=2,
)
plt.plot(
    spots,
    prices_delta_gamma_approx_millions,
    ':',
    linewidth=2.5,
    label='Delta-Gamma Approximation, using FXGamma/FXSpot (Quadratic)',
    color='#F18F01',
    alpha=0.8,
    zorder=2,
)

current_idx = spot_shocks.index(0.0)
current_spot = spots[current_idx]
current_color = '#FF4444'
marker_style = dict(s=120, color=current_color, marker='o', edgecolors='darkred', linewidths=2, zorder=5)

plt.scatter([current_spot], [prices_millions[current_idx]], label='Current Spot', **marker_style)
plt.scatter([current_spot], [prices_delta_approx_millions[current_idx]], **marker_style)
plt.scatter([current_spot], [prices_delta_gamma_approx_millions[current_idx]], **marker_style)

plt.axvline(x=current_spot, color=current_color, linestyle='--', alpha=0.3, linewidth=1.5)

plt.xlabel('Spot Price (USDJPY)', fontsize=12, fontweight='bold')
plt.ylabel('Option Premium (Million JPY)', fontsize=12, fontweight='bold')
plt.title('Option Price: Actual vs Delta & Delta-Gamma Approximations', fontsize=14, fontweight='bold', pad=20)
plt.legend(loc='best', fontsize=10, framealpha=0.9, shadow=True)
plt.grid(True, alpha=0.3, linestyle='--', linewidth=0.5)

ax = plt.gca()
ax.yaxis.set_major_formatter(FuncFormatter(lambda x, p: f'{x:.2f}'))